In [31]:
import os
# !wget https://cs.stanford.edu/group/mlgroup/MURA-v1.1.zip
from google.colab import files

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!cp /content/drive/MyDrive/MURA-v1.1.zip /content

In [5]:
!unzip MURA-v1.1.zip

Streaming output truncated to the last 5000 lines.
   creating: MURA-v1.1/train/XR_SHOULDER/patient01061/
   creating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/image1.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/image3.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01061/study1_positive/image2.png  
   creating: MURA-v1.1/train/XR_SHOULDER/patient01016/
   creating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/image1.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/image3.png  
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01016/study1_positive/image2.png  
   creating: MURA-v1.1/train/XR_SHOULDER/patient01182/
   creating: MURA-v1.1/train/XR_SHOULDER/patient01182/study1_positive/
  inflating: MURA-v1.1/train/XR_SHOULDER/patient01182/study1_positive/ima

In [26]:
########################################
# ENTER THE NAME OF THE DIRECTORY OF YOUR BONE HERE:
# One of the seven listed below:
"""
XR_ELBOW
XR_FINGER
XR_FOREARM  (Trained@52)
XR_HAND
XR_HUMERUS  (Trained@52)
XR_SHOULDER (Trained@20)
XR_WRIST
""" 
training_bone = 'XR_FINGER'
########################################

In [33]:
saved_models_path = "/content/drive/MyDrive/saved_models_"+training_bone
os.makedirs(saved_models_path, exist_ok=True)
os.makedirs("valid", exist_ok=True)
os.makedirs("train", exist_ok=True)
os.makedirs(saved_models_path+"/log", exist_ok=True)
os.makedirs("figures", exist_ok=True)


In [28]:
os.system("cp -r /content/MURA-v1.1/train/"+training_bone+" train/")
os.system("cp -r /content/MURA-v1.1/valid/"+training_bone+" valid/")

0

In [29]:
!wget 'https://raw.githubusercontent.com/AhmedSamySaad/MAC/master/densenet.py' -O densenet.py
!wget 'https://raw.githubusercontent.com/AhmedSamySaad/MAC/master/data_loader.py' -O data_loader.py

--2021-12-22 12:01:50--  https://raw.githubusercontent.com/AhmedSamySaad/MAC/master/densenet.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6999 (6.8K) [text/plain]
Saving to: ‘densenet.py’

densenet.py         100%[===================>]   6.83K  --.-KB/s    in 0s      

2021-12-22 12:01:50 (38.0 MB/s) - ‘densenet.py’ saved [6999/6999]

--2021-12-22 12:01:51--  https://raw.githubusercontent.com/AhmedSamySaad/MAC/master/data_loader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2097 (2.0K) [text/plain]
Saving to: ‘data_load

In [ ]:
from __future__ import print_function

import os
import time
import datetime
import random
import json
import argparse
import densenet
import numpy as np
import keras.backend as K

from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils

import data_loader
import re




def run_MURA(batch_size, nb_epoch, depth, nb_dense_block, nb_filter, growth_rate, dropout_rate, learning_rate, weight_decay, plot_architecture):
    """
    Run MURA experiments

    :parameter batch_size: int -- batch size
    :parameter nb_epoch: int -- number of training epochs
    :parameter depth: int -- network depth
    :parameter nb_dense_block: int -- number of dense blocks
    :parameter nb_filter: int -- initial number of conv filter
    :parameter growth_rate: int -- number of new filters added by conv layers
    :parameter dropout_rate: float -- dropout rate
    :parameter learning_rate: float -- learning rate
    :parameter weight_decay: float -- weight decay
    :parameter plot_architecture: bool -- whether to plot network architecture

    """

    ###################
    # Data processing #
    ###################

    
    im_size = 320   #Test modification parameters size root_path nb_epoch nb_dense_block         
    X_train_path, Y_train = data_loader.load_path(root_path = './train/'+training_bone,size = im_size)  
    X_valid_path, Y_valid = data_loader.load_path(root_path = './valid/'+training_bone, size = im_size)  

    X_valid = data_loader.load_image(X_valid_path,im_size)  #Load verification set ahead of time
    Y_valid = np.asarray(Y_valid)
    nb_classes = 1                                
    img_dim = (im_size,im_size,1)                 #Plus the last dimension, type is tuple

    
    ###################
    # Construct model #
    ###################

    ##ADDED THIS TO RESUME TRAINING
    if os.path.isdir(saved_models_path):
         path = []
         for root,dirs,files in os.walk(saved_models_path): #Read all pictures, os.walk Return to iterator genertor Traverse all files
		          for name in files:
			             path_1 = os.path.join(root,name)
			             path.append(path_1)
         if len(path)> 0:
              model= densenet.DenseNet(nb_classes, img_dim, depth, nb_dense_block, growth_rate, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)
              last_weights= path[-1]
              model.load_weights(last_weights)
              start= last_weights.split('/')[-1] #to determine the last epoch trained
              start= int(re.search(r'\d+', start).group())
         else:
          # print('lllllllllllllllllllllllllll')
              model = densenet.DenseNet(nb_classes, img_dim, depth, nb_dense_block, growth_rate, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)
              start= 0
    
    model.summary()
    # Build optimizer
    opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

    if plot_architecture:
        from keras.utils import plot_model
        plot_model(model, to_file='./figures/densenet_archi.png', show_shapes=True)

    ####################
    # Network training #
    ####################

    print("Start Training")

    list_train_loss = []
    list_valid_loss = []
    list_learning_rate = []
    best_record = [100,0,100,100] #Recording optimal [verification set loss function value, accuracy rate, training set data set loss difference,acc difference]
    start_time = datetime.datetime.now()
    for e in range(start, nb_epoch):

        if e == int(0.25 * nb_epoch):
            K.set_value(model.optimizer.lr, np.float32(learning_rate / 10.))

        if e == int(0.5 * nb_epoch):
            K.set_value(model.optimizer.lr, np.float32(learning_rate / 50.))

        if e == int(0.75 * nb_epoch):
            K.set_value(model.optimizer.lr, np.float32(learning_rate / 100.))


        split_size = batch_size
        num_splits = len(X_train_path) / split_size

        arr_all = np.arange(len(X_train_path)).astype(int)
        random.shuffle(arr_all)                 #Randomly disrupted index order
        arr_splits = np.array_split(arr_all, num_splits)

        l_train_loss = []
        batch_train_loss = []
        start = datetime.datetime.now()

        for i,batch_idx in enumerate(arr_splits):


            X_batch_path,Y_batch = [],[]
            for idx in batch_idx:
                X_batch_path.append(X_train_path[idx])
                Y_batch.append(Y_train[idx])
            X_batch = data_loader.load_image(Path = X_batch_path, size =im_size)
            Y_batch = np.asarray(Y_batch)
            train_logloss, train_acc = model.train_on_batch(X_batch, Y_batch)

            l_train_loss.append([train_logloss, train_acc])
            batch_train_loss.append([train_logloss, train_acc])
            if i %100 == 0:
                loss_1, acc_1 = np.mean(np.array(l_train_loss), 0)
                loss_2, acc_2 = np.mean(np.array(batch_train_loss), 0)
                batch_train_loss = []           #Current 100 batch loss function and accuracy
                print ('[Epoch {}/{}] [Batch {}/{}] [Time: {}] [all_batchs--> train_epoch_logloss: {:.5f}, train_epoch_acc:{:.5f}] '.format(e+1,nb_epoch,i, len(arr_splits),datetime.datetime.now() - start,loss_1,acc_1), '[this_100_batchs-->train_batchs_logloss: {:.5f}, train_batchs_acc:{:.5f}]'.format(loss_2, acc_2))

        # Run verification set
        valid_logloss, valid_acc = model.evaluate(X_valid, Y_valid, verbose=0, batch_size=64)
        list_train_loss.append(np.mean(np.array(l_train_loss), 0).tolist())
        list_valid_loss.append([valid_logloss, valid_acc])
        list_learning_rate.append(float(K.get_value(model.optimizer.lr)))
        # to convert numpy array to json serializable
        print('[Epoch %s/%s] [Time: %s, Total_time: %s]' % (e + 1, nb_epoch, datetime.datetime.now() - start, datetime.datetime.now() - start_time),end = '')
        print('[train_loss_and_acc:{:.5f} {:.5f}] [valid_loss_acc:{:.5f} {:.5f}]'.format(list_train_loss[-1][0], list_train_loss[-1][1],list_valid_loss[-1][0],list_valid_loss[-1][1]))


        d_log = {}
        d_log["batch_size"] = batch_size
        d_log["nb_epoch"] = nb_epoch
        d_log["optimizer"] = opt.get_config()
        d_log["train_loss"] = list_train_loss
        d_log["valid_loss"] = list_valid_loss
        d_log["learning_rate"] = list_learning_rate

        # json_file = os.path.join('./log/experiment_log_MURA.json')
        json_file = saved_models_path+"/log/experiment_log_MURA.json"
        with open(json_file, 'w') as fp:
            json.dump(str(d_log), fp, indent=4, sort_keys=True)



        record = [valid_logloss,valid_acc,abs(valid_logloss-list_train_loss[-1][0]),abs(valid_acc-list_train_loss[-1][1]),]
        if ((record[0]<=best_record[0]) &(record[1]>=best_record[1])) :
            if e <= int(0.25 * nb_epoch)|(record[2]<=best_record[2])&(record[3]<=best_record[3]):#Add a difference judgment after a quarter epoch
                best_record=record                      #Record the smallest [validation set loss function value, accuracy rate, training set data loss difference, acc difference]
                print('saving the best model:epoch',e+1,best_record)
                best_model_weight= saved_models_path+'/best_MURA_model@epochs{}.h5'.format(e+1)
                model.save(best_model_weight)
                # !cp /content/save_models/best_model_weight /content/drive/MyDrive/saved_model
                # files.download(best_model_weight)
        weight_file= saved_models_path+'/MURA_model@epochs{}.h5'.format(e+1)
        model.save(weight_file)

run_MURA(batch_size= 8, nb_epoch= 52, depth= 22, nb_dense_block= 4, nb_filter= 16, growth_rate= 12, dropout_rate= 0.2, learning_rate=1E-3, weight_decay=1E-4, plot_architecture= False)

5106
461
Model: "DenseNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 320, 320, 1  0           []                               
                                )]                                                                
                                                                                                  
 initial_conv2D (Conv2D)        (None, 160, 160, 16  784         ['input_6[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_840 (Batch  (None, 160, 160, 16  64         ['initial_conv2D[0][0]']         
 Normalization)                 )                                                 